In [63]:
import pandas as pd
import numpy as np

from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

KEY = "PKN99TWE381RXZFC5P04"
SECRET_KEY = "e6kwHgPJoGr9sanjK2uTmqTynQfg7tvQXBMgR6lW"

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

#### Stock predictions

In [64]:
df = pd.read_csv("files/predictions.csv", index_col="ticker")
ticks = df.index.to_list()

#### Tradeable and shortable stocks

In [65]:
assets = trading_client.get_all_assets()

assets = [x for x in assets if (x.asset_class[:]=='us_equity') and (x.symbol in ticks) and (x.status[:]=='active')]
symbols = [x.symbol for x in assets]
tradable = [x.tradable for x in assets]
shortable = [x.shortable for x in assets]

tradable = pd.Series(tradable, index=symbols)
shortable = pd.Series(shortable, index=symbols)


df["tradable"] = tradable
df["shortable"] = shortable

df = df[~df.tradable.isna() & df.tradable]
ticks = df.index.to_list()

#### Compute trades

In [66]:

params = StockLatestQuoteRequest(symbol_or_symbols=ticks)
quotes = data_client.get_stock_latest_quote(params)
df["ask"] = [quotes[x].ask_price for x in ticks]
df["bid"] = [quotes[x].bid_price for x in ticks]

equity = 100000
numstocks = 200
longdollars = 1.3*equity / numstocks
shortdollars = 0.3*equity / numstocks

df["qlong"] = [int(longdollars/x) if x>0 else np.nan for x in df.ask]
df["qshort"] = [int(shortdollars/x) if x>0 else np.nan for x in df.bid]

df = df.sort_values(by="predict")
best = df[df.qlong>0].iloc[:200]
worst = df[(df.qshort>0) & df.shortable].iloc[-200:]

#### Submit trades

In [67]:
for tick in worst.index: 
    try:
        market_order_data = MarketOrderRequest(
            symbol=tick,
            qty=worst.loc[tick, "qshort"],
            side=OrderSide.SELL,
            time_in_force=TimeInForce.DAY
        )
        market_order = trading_client.submit_order(
            order_data=market_order_data
        )
    except:
        print(f"sell order for {tick} failed")
        
for tick in best.index: 
    try:
        market_order_data = MarketOrderRequest(
            symbol=tick,
            qty=best.loc[tick, "qlong"],
            side=OrderSide.BUY,
            time_in_force=TimeInForce.DAY
        )
        market_order = trading_client.submit_order(
            order_data=market_order_data
        )
    except:
        print(f"buy order for {tick} failed")

